In [ ]:
# def iou(box1_list, box2_list):

#     """

#     Calculates Intersection over Union for two bounding boxes (xmin, ymin, xmax, ymax)

#     returns IoU vallue

#     """


#     box1 ={}

#     box1['xmin'] = box1_list[0]

#     box1['ymin'] = box1_list[1]

#     box1['xmax'] = box1_list[2]

#     box1['ymax'] = box1_list[3]


#     box2 ={}

#     box2['xmin'] = box2_list[0]

#     box2['ymin'] = box2_list[1]

#     box2['xmax'] = box2_list[2]

#     box2['ymax'] = box2_list[3]


#     #corrdinates of intersection

#     int_x_min = max(box1['xmin'], box2['xmin'])

#     int_x_max = min(box1['xmax'], box2['xmax'])

#     int_y_min = max(box1['ymin'], box2['ymin'])

#     int_y_max = min(box1['ymax'], box2['ymax'])


#     if((int_x_max<int_x_min) or (int_y_max<int_y_min)):

#         return 0.0


#     int_area = (int_y_max-int_y_min)*(int_x_max-int_x_min) 


#     area_box1 = (box1['xmax'] - box1['xmin']) * (box1['ymax'] - box1['ymin'])

#     area_box2 = (box2['xmax'] - box2['xmin']) * (box2['ymax'] - box2['ymin'])


#     iou = int_area / float(area_box1+area_box2-int_area)


#     if(iou<0):

#         iou = 0

#     if(iou>1):

#         iou = 1


#     return iou


In [43]:
def calc_intersection_area(box1_list, box2_list):
    box1 ={}
    box1['xmin'] = box1_list[0]
    box1['ymin'] = box1_list[1]
    box1['xmax'] = box1_list[2]
    box1['ymax'] = box1_list[3]

    box2 ={}
    box2['xmin'] = box2_list[0]
    box2['ymin'] = box2_list[1]
    box2['xmax'] = box2_list[2]
    box2['ymax'] = box2_list[3]

    #corrdinates of intersection
    int_x_min = max(box1['xmin'], box2['xmin'])
    int_x_max = min(box1['xmax'], box2['xmax'])
    int_y_min = max(box1['ymin'], box2['ymin'])
    int_y_max = min(box1['ymax'], box2['ymax'])

    if((int_x_max<int_x_min) or (int_y_max<int_y_min)):
        return 0.0

    int_area = (int_y_max-int_y_min)*(int_x_max-int_x_min) 
    
    return int_area

def calc_union_area(box1_list, box2_list):
    box1 ={}
    box1['xmin'] = box1_list[0]
    box1['ymin'] = box1_list[1]
    box1['xmax'] = box1_list[2]
    box1['ymax'] = box1_list[3]

    box2 ={}
    box2['xmin'] = box2_list[0]
    box2['ymin'] = box2_list[1]
    box2['xmax'] = box2_list[2]
    box2['ymax'] = box2_list[3]

    #corrdinates of intersection
    int_x_min = max(box1['xmin'], box2['xmin'])
    int_x_max = min(box1['xmax'], box2['xmax'])
    int_y_min = max(box1['ymin'], box2['ymin'])
    int_y_max = min(box1['ymax'], box2['ymax'])

    if((int_x_max<int_x_min) or (int_y_max<int_y_min)):
        return 0.0

    int_area = (int_y_max-int_y_min)*(int_x_max-int_x_min) 
    area_box1 = (box1['xmax'] - box1['xmin']) * (box1['ymax'] - box1['ymin'])
    area_box2 = (box2['xmax'] - box2['xmin']) * (box2['ymax'] - box2['ymin'])
    
    return float(area_box1+area_box2-int_area)

In [63]:
import json
import glob
import numpy as np
from tqdm import tqdm

# Available files

In [7]:
with open("finaldata_gt.json") as f:
    json_gt = json.load(f)
    
completed_list = []
for name in glob.glob('layout_completed/*.json'):
    num = int(name.split("/")[1].split(".")[0])
    completed_list.append(num)
    
gt_list = []
for name in glob.glob('layout_gt/*.json'):
    num = int(name.split("/")[1].split(".")[0])
    gt_list.append(num)
    
missing_list = []
for name in glob.glob('layout_missing/*.json'):
    num = int(name.split("/")[1].split(".")[0])
    missing_list.append(num)

In [8]:
print(len(json_gt))
print(len(completed_list))
print(len(gt_list))
print(len(missing_list))


3596
2811
920
2966


# Calculating for all 

In [58]:
gt_iou_dict = {}
completed_iou_dict = {}
missing_iou_dict = {}


for obj in tqdm(json_gt, total = len(json_gt)):
    id_obj = obj['id']
    if((id_obj in completed_list) & (id_obj in gt_list) & (id_obj in missing_list)):
        
        height = obj['height']
        width = obj['width']
        
        with open('layout_completed/' + str(id_obj) + '.json') as f:
            completed_json = json.load(f)
            completed_obj_dict = {}
            if(len(completed_json['classes']) == 1):
                completed_json['boxes'] = [completed_json['boxes']]
            for box, clas in zip(completed_json['boxes'], completed_json['classes']):
                if clas in completed_obj_dict.keys():
                    completed_obj_dict[clas].append(box)
                else:
                    completed_obj_dict[clas] = []
                    completed_obj_dict[clas].append(box)
            
        with open('layout_gt/' + str(id_obj) + '.json') as f:
            gt_json = json.load(f)
            gt_obj_dict = {}
            if(len(gt_json['classes']) == 1):
                gt_json['boxes'] = [gt_json['boxes']]
            for box, clas in zip(gt_json['boxes'], gt_json['classes']):
                if clas in gt_obj_dict.keys():
                    gt_obj_dict[clas].append(box)
                else:
                    gt_obj_dict[clas] = []
                    gt_obj_dict[clas].append(box)
            
        with open('layout_missing/' + str(id_obj) + '.json') as f:
            missing_json = json.load(f)
            missing_obj_dict = {}
            if(len(missing_json['classes']) == 1):
                missing_json['boxes'] = [missing_json['boxes']]
            for box, clas in zip(missing_json['boxes'], missing_json['classes']):
                if clas in missing_obj_dict.keys():
                    missing_obj_dict[clas].append(box)
                else:
                    missing_obj_dict[clas] = []
                    missing_obj_dict[clas].append(box)
                    
        
        gt_num = 0
        gt_denum = 0
        completed_num = 0
        completed_denum = 0
        missing_num = 0
        missing_denum = 0
        
        
        for gt_obj in obj['objects']:
            box = gt_obj['box']
            box = [box[0]/width, box[1]/height, box[2]/width, box[3]/height]
            clas = gt_obj['class']
            box_area = (box[3]-box[1])*(box[2]-box[0])
            
            #for gt
            if clas in gt_obj_dict.keys():
                best_overlap = 0
                best_pos = None
                
                for coords in gt_obj_dict[clas]:
                    int_area = calc_intersection_area(coords, box)
                    if(int_area>=best_overlap):
                        best_overlap = int_area
                        best_pos = coords
                        
                gt_obj_dict[clas].remove(best_pos)
                if(len(gt_obj_dict[clas])==0):
                    del gt_obj_dict[clas]
                gt_num += calc_intersection_area(best_pos, box)
                gt_denum += calc_union_area(best_pos, box)  
            else:
                gt_denum += box_area
                
            #for completed
            if clas in completed_obj_dict.keys():
                best_overlap = 0
                best_pos = None
                
                for coords in completed_obj_dict[clas]:
                    int_area = calc_intersection_area(coords, box)
                    if(int_area>=best_overlap):
                        best_overlap = int_area
                        best_pos = coords
                        
                completed_obj_dict[clas].remove(best_pos)
                if(len(completed_obj_dict[clas])==0):
                    del completed_obj_dict[clas]
                completed_num += calc_intersection_area(best_pos, box)
                completed_denum += calc_union_area(best_pos, box)  
            else:
                completed_denum += box_area
                
            
            #for missing
            if clas in missing_obj_dict.keys():
                best_overlap = 0
                best_pos = None
                
                for coords in missing_obj_dict[clas]:
                    int_area = calc_intersection_area(coords, box)
                    if(int_area>=best_overlap):
                        best_overlap = int_area
                        best_pos = coords
                        
                missing_obj_dict[clas].remove(best_pos)
                if(len(missing_obj_dict[clas])==0):
                    del missing_obj_dict[clas]
                missing_num += calc_intersection_area(best_pos, box)
                missing_denum += calc_union_area(best_pos, box)  
            else:
                missing_denum += box_area
                
        if(gt_denum>0):
            gt_iou_dict[id_obj] = float(gt_num/gt_denum)
        else:
            gt_iou_dict[id_obj] = 0
            
        if(completed_denum>0):
            completed_iou_dict[id_obj] = float(completed_num/completed_denum)
        else:
            completed_iou_dict[id_obj] = 0
            
        if(missing_denum>0):
            missing_iou_dict[id_obj] = float(missing_num/missing_denum)
        else:
            missing_iou_dict[id_obj] = 0
          

100%|██████████| 3596/3596 [00:00<00:00, 9906.02it/s] 


In [66]:
print(np.mean(np.array(list(gt_iou_dict.values()))))
print(np.mean(np.array(list(completed_iou_dict.values()))))
print(np.mean(np.array(list(missing_iou_dict.values()))))

0.2605072272175019
0.2340785063244288
0.23740863438627663


In [71]:
missing_iou_dict

{1: 0.29237124525015673,
 7: 0.3283328357343947,
 8: 0.1062603920344638,
 11: 0.18023705520206973,
 12: 0.319805031944766,
 15: 0.13284533083747568,
 25: 0.1713883950055886,
 26: 0.5227553678342178,
 31: 0.1428115651052178,
 33: 0.18414072728013656,
 34: 0.2694054452567415,
 36: 0.1227164748491181,
 37: 0.22154255191190192,
 41: 0.7619829690330966,
 44: 0.36605345674597245,
 45: 0.19581987187182046,
 46: 0.4075803409374468,
 48: 0.05644323354848726,
 56: 0.056826925447032335,
 59: 0.04359024206754369,
 60: 0.31248580280887484,
 61: 0.2217723622111121,
 63: 0.18644864675481093,
 77: 0.2923608257558948,
 81: 0.32643137347697904,
 91: 0.5012388674692106,
 97: 0.36124418297351213,
 101: 0.40513110952386694,
 102: 0.14221067241500687,
 106: 0.07643466646788173,
 116: 0.13247695263532616,
 118: 0.1343894437451277,
 120: 0.13660946241209354,
 121: 0.19776750158619036,
 123: 0.41945618619415037,
 124: 0.13108704620284234,
 127: 0.280897564895755,
 130: 0.25241173386148996,
 135: 0.471953584372

In [72]:
completed_iou_dict

{1: 0.18822161538120533,
 7: 0.4982735755657225,
 8: 0.10322917255645714,
 11: 0.15916189448545537,
 12: 0.4608274275450679,
 15: 0.23613842073569452,
 25: 0.2207074053252595,
 26: 0.2883750693887737,
 31: 0.4119308364083338,
 33: 0.12278864332225,
 34: 0.08750939593978802,
 36: 0.21360339148720908,
 37: 0.32716896034756837,
 41: 0.48853628391318743,
 44: 0.44805357232245296,
 45: 0.160072141561857,
 46: 0.17588441619058268,
 48: 0.6217548598617305,
 56: 0.052896496866555065,
 59: 0.03863690751818177,
 60: 0.20790541337209148,
 61: 0.243733292730405,
 63: 0.07543339665069217,
 77: 0.052852336381576104,
 81: 0.33020536727230704,
 91: 0.18698827199624687,
 97: 0.4066621430223538,
 101: 0.3661649829695836,
 102: 0.14337770290810425,
 106: 0.10620184125754846,
 116: 0.1603754835863398,
 118: 0.42492794189438593,
 120: 0.23756827040657727,
 121: 0.19860701412144585,
 123: 0.27125333453419187,
 124: 0.19460660656105297,
 127: 0.08785419426668654,
 130: 0.18304544475785953,
 135: 0.1201044232

In [67]:
obj

{'relationships': [{'sub_id': 1, 'predicate': 'hold', 'obj_id': 0},
  {'sub_id': 3, 'predicate': 'of', 'obj_id': 2},
  {'sub_id': 4, 'predicate': 'on', 'obj_id': 2},
  {'sub_id': 2, 'predicate': 'have', 'obj_id': 5},
  {'sub_id': 6, 'predicate': 'on', 'obj_id': 2},
  {'sub_id': 1, 'predicate': 'hold', 'obj_id': 0},
  {'sub_id': 4, 'predicate': 'on', 'obj_id': 2},
  {'sub_id': 7, 'predicate': 'of', 'obj_id': 1},
  {'sub_id': 9, 'predicate': 'hold', 'obj_id': 8},
  {'sub_id': 6, 'predicate': 'of', 'obj_id': 2},
  {'sub_id': 3, 'predicate': 'of', 'obj_id': 2}],
 'objects': [{'box': [212, 141, 225, 196], 'class': 'picture'},
  {'box': [201, 105, 224, 203], 'class': 'man'},
  {'box': [126, 16, 483, 175], 'class': 'bus'},
  {'box': [309, 153, 365, 227], 'class': 'wheel'},
  {'box': [328, 64, 362, 99], 'class': 'woman'},
  {'box': [215, 60, 390, 115], 'class': 'window'},
  {'box': [403, 46, 465, 194], 'class': 'door'},
  {'box': [205, 114, 223, 136], 'class': 'helmet'},
  {'box': [286, 178, 3

In [68]:
completed_json

{'image_id': 3595,
 'boxes': [[0.38997411727905273,
   0.31653010845184326,
   0.5919907689094543,
   0.7332413792610168],
  [0.313914030790329,
   0.29993757605552673,
   0.7280913591384888,
   0.7340736985206604],
  [0.27131444215774536,
   0.2303369790315628,
   0.6665156483650208,
   0.657683789730072],
  [0.283855676651001,
   0.24910321831703186,
   0.6688773036003113,
   0.5998541712760925],
  [0.41674596071243286,
   0.3368557393550873,
   0.5446748733520508,
   0.4818150997161865],
  [0.4491800367832184,
   0.368854820728302,
   0.5243663191795349,
   0.45145654678344727],
  [0.40926748514175415,
   0.30987298488616943,
   0.5328967571258545,
   0.6118921637535095],
  [0.4510152041912079,
   0.6375890970230103,
   0.5214585661888123,
   0.7448453903198242],
  [0.4573787748813629,
   0.6419143080711365,
   0.5330630540847778,
   0.7625071406364441]],
 'classes': ['man',
  'picture',
  'bus',
  'window',
  'helmet',
  'hat',
  'door',
  'wheel',
  'tire'],
 'class_ids': [3, 8, 1

In [69]:
gt_json

{'image_id': 3595,
 'boxes': [[0.2644960582256317,
   0.16618458926677704,
   0.6774711012840271,
   0.8511987924575806],
  [0.3906124532222748,
   0.3721896708011627,
   0.5907043814659119,
   0.631036639213562],
  [0.4502510726451874,
   0.6704441905021667,
   0.5234138369560242,
   0.775272786617279],
  [0.3351067006587982,
   0.2644776403903961,
   0.7389014959335327,
   0.9175601005554199],
  [0.2934747636318207,
   0.26846110820770264,
   0.6912776231765747,
   0.9165397882461548],
  [0.2996281683444977,
   0.2706499993801117,
   0.6851691603660583,
   0.6910179257392883],
  [0.4057423174381256,
   0.3363068699836731,
   0.5713226199150085,
   0.696199357509613],
  [0.40823331475257874,
   0.3440309464931488,
   0.48972630500793457,
   0.4498537480831146],
  [0.37520551681518555,
   0.3077315390110016,
   0.5640186667442322,
   0.7783448100090027],
  [0.43685799837112427,
   0.3927571475505829,
   0.5154814720153809,
   0.48041924834251404]],
 'classes': ['man',
  'picture',
  'w

In [70]:
missing_json

{'image_id': 3595,
 'boxes': [[0.2611399292945862,
   0.14281605184078217,
   0.7138311266899109,
   0.8671814799308777],
  [0.4122796058654785,
   0.39325466752052307,
   0.6150315403938293,
   0.6275699734687805],
  [0.25837579369544983,
   0.06676643341779709,
   0.7032954692840576,
   0.7596274614334106],
  [0.45779258012771606,
   0.3337852656841278,
   0.5239988565444946,
   0.44379541277885437],
  [0.4248739778995514,
   0.29951950907707214,
   0.5431925654411316,
   0.42912039160728455],
  [0.4303603172302246,
   0.3642999231815338,
   0.5896693468093872,
   0.5289917588233948],
  [0.4482300281524658,
   0.39034730195999146,
   0.5175803899765015,
   0.5402024388313293],
  [0.4510827958583832,
   0.5692002773284912,
   0.5241646766662598,
   0.6798232793807983]],
 'classes': ['man',
  'picture',
  'bus',
  'window',
  'helmet',
  'hat',
  'door',
  'wheel'],
 'class_ids': [3, 7, 1, 6, 8, 4, 2, 5]}

# Edge case example

In [36]:
obj

{'relationships': [{'sub_id': 1, 'predicate': 'on', 'obj_id': 0},
  {'sub_id': 2, 'predicate': 'on', 'obj_id': 0},
  {'sub_id': 4, 'predicate': 'in', 'obj_id': 3},
  {'sub_id': 1, 'predicate': 'on', 'obj_id': 0},
  {'sub_id': 2, 'predicate': 'on', 'obj_id': 0},
  {'sub_id': 5, 'predicate': 'on', 'obj_id': 1},
  {'sub_id': 4, 'predicate': 'in', 'obj_id': 3},
  {'sub_id': 5, 'predicate': 'of', 'obj_id': 1},
  {'sub_id': 7, 'predicate': 'have', 'obj_id': 6},
  {'sub_id': 7, 'predicate': 'have', 'obj_id': 8},
  {'sub_id': 1, 'predicate': 'on', 'obj_id': 0},
  {'sub_id': 0, 'predicate': 'next_to', 'obj_id': 7},
  {'sub_id': 5, 'predicate': 'on', 'obj_id': 1},
  {'sub_id': 6, 'predicate': 'in', 'obj_id': 7}],
 'objects': [{'box': [237, 6, 358, 159], 'class': 'tower'},
  {'box': [253, 51, 302, 106], 'class': 'clock'},
  {'box': [312, 52, 346, 112], 'class': 'clock'},
  {'box': [0, 1, 497, 368], 'class': 'sky'},
  {'box': [1, 4, 499, 200], 'class': 'cloud'},
  {'box': [270, 61, 287, 87], 'clas

In [39]:
completed_json

{'image_id': 1,
 'boxes': [[0.43163442611694336,
   0.44313088059425354,
   0.5098111033439636,
   0.5164026618003845],
  [0.4542318284511566,
   0.4164986312389374,
   0.6205577254295349,
   0.6723538041114807],
  [0.373328298330307, 0.274645060300827, 0.5875260233879089, 0.37408646941185],
  [0.01177306193858385,
   0.09966649860143661,
   0.9512948989868164,
   0.5459114909172058],
  [0.16666054725646973,
   0.1724604219198227,
   0.7668628096580505,
   0.6723689436912537],
  [0.43077754974365234,
   0.3843236565589905,
   0.5757657289505005,
   0.5052660703659058],
  [0.4517997205257416,
   0.483351469039917,
   0.49508777260780334,
   0.518878161907196],
  [0.39023956656455994,
   0.31467023491859436,
   0.4953647553920746,
   0.37203970551490784]],
 'classes': ['clock',
  'tower',
  'cloud',
  'sky',
  'building',
  'window',
  'hand',
  'roof'],
 'class_ids': [6, 8, 5, 2, 1, 4, 7, 3]}

In [16]:
gt_json

{'image_id': 1,
 'boxes': [[0.36965829133987427,
   0.3578331470489502,
   0.5808011293411255,
   0.5380578637123108],
  [0.42951321601867676,
   0.4210330545902252,
   0.6426665782928467,
   0.6135338544845581],
  [0.02392282523214817,
   0.20425350964069366,
   0.31380876898765564,
   0.47510385513305664],
  [0.0015625000232830644,
   0.05877583101391792,
   0.9284178614616394,
   0.4274601936340332],
  [0.04213099554181099,
   0.05928394943475723,
   0.9869896173477173,
   0.5294995307922363],
  [0.4101649224758148,
   0.4349629878997803,
   0.4776735305786133,
   0.4953877031803131],
  [0.0015625000232830644,
   0.13993845880031586,
   0.8968569040298462,
   0.7626785635948181],
  [0.39089444279670715,
   0.3955479860305786,
   0.5699443221092224,
   0.5351486802101135],
  [0.13292519748210907,
   0.3152633607387543,
   0.20733682811260223,
   0.3938397467136383]],
 'classes': ['clock',
  'tower',
  'clock',
  'cloud',
  'sky',
  'hand',
  'building',
  'window',
  'window'],
 'cla

In [17]:
missing_json

{'image_id': 1,
 'boxes': [[0.3852786123752594,
   0.3864447772502899,
   0.5667931437492371,
   0.5366503596305847],
  [0.43256112933158875,
   0.3778783977031708,
   0.6265701651573181,
   0.6551405787467957],
  [0.36592885851860046,
   0.23429112136363983,
   0.6161720156669617,
   0.37061363458633423],
  [0.048485469073057175,
   0.07050301879644394,
   0.9887507557868958,
   0.5759978294372559],
  [0.19393105804920197,
   0.13249872624874115,
   0.6046408414840698,
   0.6031271815299988],
  [0.31706684827804565,
   0.290985643863678,
   0.49059462547302246,
   0.4440270960330963],
  [0.418790340423584,
   0.4676856994628906,
   0.47720012068748474,
   0.5205820202827454]],
 'classes': ['clock', 'tower', 'cloud', 'sky', 'building', 'window', 'hand'],
 'class_ids': [5, 7, 4, 2, 1, 3, 6]}